In [ ]:
# %%capture
# !pip install git+https://github.com/biothings/biothings_explorer#egg=biothings_explorer

In [94]:
import os
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import date
import datetime as dt
import pandas as pd
import sys
import csv
import numpy as np
import multiprocessing

In [95]:
import psycopg2
import pandas.io.sql as sqlio
# from tabulate import tabulate

In [96]:
import json
import argparse
import lxml
import config

# from Authentication import *
import requests
import json
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry


In [ ]:
url = "https://aact.ctti-clinicaltrials.org/pipe_files"
response = requests.get(url)

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.text)
# print(soup.prettify())

In [ ]:

# https://aact.ctti-clinicaltrials.org/static/exported_files/monthly/20220301_pipe-delimited-export.zip
# https://aact.ctti-clinicaltrials.org/static/exported_files/monthly/20220801_pipe-delimited-export.zip
upload_dates = []
zip_files = []
links = []
body = soup.find_all('td', attrs={'class': 'file-archive'}) #Find all
for el in body:
    tags = el.find('a')
    try:
        if 'href' in tags.attrs:   # looking for href inside anchor tag    
            link = "https://aact.ctti-clinicaltrials.org" + tags.get('href')
            links.append(link)
            last_upload = link.split("/")[-1]
            zip_files.append(last_upload)
            date_upload = last_upload.split("_")[0]
            upload_dates.append(date_upload)    # appending link to list of links
    except:    # pass if list missing anchor tag or anchor tag does not has a href params 
        pass
    
print(upload_dates)

upload_dates = [dt.datetime.strptime(date, '%Y%m%d').date() for date in upload_dates] # convert all strings in list into datetime objects


In [ ]:
most_recent_date = max(upload_dates)
# print(most_recent_date)

today = dt.date.today()

# UNCOMMENT LINE BELOW WHEN IN PRODUCTION, THIS IS FOR TESTING PURPOSES
most_recent_date = min(upload_dates)

# fix this to run entire script should the current date be 
if most_recent_date < today:
#     date_file_df = pd.DataFrame(list(zip(upload_dates, zip_files, links)))
#     print(links[0])
    resp = urlopen(links[0])
else:
    print("Local instance of clinical trials data is already up to date.")



In [ ]:
pd.set_option('display.max_colwidth', 3000)
# date_file_df[2]

In [ ]:
os.getcwd()


In [97]:
# connect to DB and get the column names of the table
con = None
con = psycopg2.connect(database="aact")
con.rollback()
cursor = con.cursor()

con.autocommit = True # SQL statement is treated as a transaction and is automatically committed right after it is executed
# grab the conditions
sql = '''SELECT * FROM ctgov.conditions;'''
cursor.execute(sql)
column_names = [desc[0] for desc in cursor.description]
tuples = cursor.fetchall()
conditions_df = pd.DataFrame(tuples, columns=column_names)

#grab the browse_conditions
sql = '''SELECT * FROM ctgov.browse_conditions;'''
cursor.execute(sql)
column_names = [desc[0] for desc in cursor.description]
tuples = cursor.fetchall()
browse_conditions_df = pd.DataFrame(tuples, columns=column_names)

#grab the interventions
sql = '''SELECT * FROM ctgov.interventions;'''
cursor.execute(sql)
column_names = [desc[0] for desc in cursor.description]
tuples = cursor.fetchall()
interventions_df = pd.DataFrame(tuples, columns=column_names)

#grab the browse_interventions
sql = '''SELECT * FROM ctgov.browse_interventions;'''
cursor.execute(sql)
column_names = [desc[0] for desc in cursor.description]
tuples = cursor.fetchall()
browse_interventions_df = pd.DataFrame(tuples, columns=column_names)

con.close()


In [98]:
interventions_df = interventions_df.rename(columns={'id': 'int_id',
                                                    'nct_id': 'int_nctid',
                                                    'intervention_type': 'int_type',
                                                    'name': 'int_name',
                                                    'description': 'int_description'})
interventions_df

,int_id,int_nctid,int_type,int_name,int_description
0,4547568,NCT01340339,Device,Phototherapy,Fluorescent reverse phototherapy (7 white ligh...
1,4547569,NCT01340339,Device,Phototherapy,super LED reverse phototherapy (17 bulbs arran...
2,4547570,NCT01340365,Behavioral,Tai Chi Exercise,Practicing Tai Chi exercise 4 times a week for...
3,4547571,NCT01340365,Behavioral,Tai Chi,"Usual care, individuals attend testing session..."
4,4547572,NCT01340391,Device,Omnicast,dorsal splint 2-5 weeks
...,...,...,...,...,...
685271,4139681,NCT03104712,Other,Rice + Pesto,50g available carbohydrate
685272,4139682,NCT03104465,Behavioral,Mindfulness Training,interactive. web-based mindfulness training co...
685273,4139683,NCT03104361,Biological,Platelet-Rich Plasma,Patients who meet all eligible requirements fo...
685274,4139684,NCT03104283,Drug,Apatinib,take apatinib orally until disease progression...


In [99]:
interventions_df = interventions_df.drop(columns=['int_id', 'int_description'])

In [100]:
conditions_df = conditions_df.rename(columns={'id': 'con_id',
                                              'nct_id': 'con_nctid',
                                              'name': 'con_name',
                                              'downcase_name': 'con_downcase_name'})
conditions_df

,con_id,con_nctid,con_name,con_downcase_name
0,3072834,NCT03254264,Autism Spectrum Disorder,autism spectrum disorder
1,3072835,NCT03254329,Human Milk Nutrient Reference Values,human milk nutrient reference values
2,3349485,NCT03460652,ADHD,adhd
3,3349486,NCT03460899,Diabetes Mellitus With Hypoglycemia,diabetes mellitus with hypoglycemia
4,3072836,NCT03254342,Major Depressive Disorder,major depressive disorder
...,...,...,...,...
688783,3072052,NCT03261687,Pregnancy Related,pregnancy related
688784,3072053,NCT03261687,Pelvic Girdle Pain,pelvic girdle pain
688785,3072054,NCT03261622,Faecal Incontinence,faecal incontinence
688786,3072055,NCT03261622,Fecal Incontinence,fecal incontinence


In [101]:
conditions_df = conditions_df.drop(columns=['con_id', 'con_name'])

In [102]:
browse_interventions_df = browse_interventions_df.rename(columns={'id': 'browseint_id',
                                                                  'nct_id': 'browseint_nctid',
                                                                  'mesh_term': 'browseint_meshterm',
                                                                  'downcase_mesh_term': 'browseint_meshterm_downcase',
                                                                  'mesh_type': 'browseint_meshtype'})
browse_interventions_df

,browseint_id,browseint_nctid,browseint_meshterm,browseint_meshterm_downcase,browseint_meshtype
0,4935968,NCT04327843,Haloperidol,haloperidol,mesh-list
1,4935969,NCT04327843,Haloperidol decanoate,haloperidol decanoate,mesh-list
2,4935970,NCT04327843,Antiemetics,antiemetics,mesh-ancestor
3,5198960,NCT00808223,Alefacept,alefacept,mesh-list
4,5198961,NCT00808223,Dermatologic Agents,dermatologic agents,mesh-ancestor
...,...,...,...,...,...
1335262,4400668,NCT03100786,"Antineoplastic Agents, Hormonal","antineoplastic agents, hormonal",mesh-ancestor
1335263,4400669,NCT03100786,Antineoplastic Agents,antineoplastic agents,mesh-ancestor
1335264,4400670,NCT03100565,Lidocaine,lidocaine,mesh-list
1335265,4400671,NCT03100565,"Anesthetics, Local","anesthetics, local",mesh-ancestor


In [103]:
browse_interventions_df = browse_interventions_df.drop(columns=['browseint_id', 'browseint_meshterm'])

In [104]:
browse_conditions_df = browse_conditions_df.rename(columns={'id': 'browsecon_id',
                                                            'nct_id': 'browsecon_nctid',
                                                            'mesh_term': 'browsecon_meshterm',
                                                            'downcase_mesh_term': 'browsecon_meshterm_downcase',
                                                            'mesh_type': 'browsecon_meshtype'})
browse_conditions_df

,browsecon_id,browsecon_nctid,browsecon_meshterm,browsecon_meshterm_downcase,browsecon_meshtype
0,9631397,NCT04035564,Hyponatremia,hyponatremia,mesh-list
1,9631398,NCT04035564,Water-Electrolyte Imbalance,water-electrolyte imbalance,mesh-ancestor
2,9631399,NCT04035564,Metabolic Diseases,metabolic diseases,mesh-ancestor
3,9631400,NCT04032652,Colitis,colitis,mesh-list
4,9631401,NCT04032652,"Colitis, Ulcerative","colitis, ulcerative",mesh-list
...,...,...,...,...,...
2573556,8601072,NCT03213899,Disease Attributes,disease attributes,mesh-ancestor
2573557,8601073,NCT03213899,Pathologic Processes,pathologic processes,mesh-ancestor
2573558,8601074,NCT03214016,Hypertension,hypertension,mesh-list
2573559,8601075,NCT03214016,Vascular Diseases,vascular diseases,mesh-ancestor


In [105]:
browse_conditions_df = browse_conditions_df.drop(columns=['browsecon_id', 'browsecon_meshterm'])

In [13]:
print(len(interventions_df['int_name'].unique()))
print(len(conditions_df['con_downcase_name'].unique()))
print(len(browse_conditions_df['browsecon_meshterm_downcase'].unique()))
print(len(browse_interventions_df['browseint_meshterm_downcase'].unique()))

350361
95250
4280
3744


In [106]:
conditions_df


,con_nctid,con_downcase_name
0,NCT03254264,autism spectrum disorder
1,NCT03254329,human milk nutrient reference values
2,NCT03460652,adhd
3,NCT03460899,diabetes mellitus with hypoglycemia
4,NCT03254342,major depressive disorder
...,...,...
688783,NCT03261687,pregnancy related
688784,NCT03261687,pelvic girdle pain
688785,NCT03261622,faecal incontinence
688786,NCT03261622,fecal incontinence


In [107]:
df = pd.merge(conditions_df, interventions_df, left_on='con_nctid', right_on = 'int_nctid')

In [ ]:
df.shape

In [108]:
df_dedup = df.drop_duplicates(subset = ['con_downcase_name', 'int_name'],
                                      keep = 'first').reset_index(drop = True)

In [ ]:
df_dedup.shape

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_dedup.head(1000))

In [ ]:
df_dedup[['con_downcase_name', 'int_name']].to_csv('ClinTrials_KG_edges.csv', sep ='\t', index=False)


# notes on creating edge file

go to /proj/arivale/gglusman/KGs

once there look into wellness/source/wellness_kg_edges_v1.6.tsv

notice the first line: subject	predicate	object	relation	subject_name	object_name	category	N	Type_of_relationship	Strength_of_relationship	qualifiers	qualifier_value	Bonferroni_pval

subject, predicate, object are required (maybe some others, don’t remember)

…and they should be CURIEs

once we map concepts to biolink, mesh etc we’ll have curies

meanwhile, the content you have fits subject_name, object_name

the predicate is what we made up… let’s say ‘evaluated_for’

for short

we’ll have to find a suitable biolink predicate, or suggest a new one

so interventions would be subjects, and conditions would be objects

looking at the several first lines in your file, I see some that make sense, others that seem to suggest there are data problems

for example, “metastatic lung cancer	Pathways” -> Pathways doesn’t seem to be an intervention

‘placebo’ we’ll have to suppress

make sure to include the NCT id in a (non-required) column

it’s essentially part of the provenance

we’ll want to add multiple additional columns with things like FDA approval status, sample size, etc etc etc

look also into EHR/source/ehr_risk_kg_edges_2021_05_07.tsv

…and of course replace ‘edges’ -> ‘nodes’ for their cognate nodes files

wellness/source/wellness_kg_edges_v1.6.tsv -> wellness/source/wellness_kg_nodes_v1.6.tsv

drop "control"

‘control’ is clear, ‘control group’ too… but ‘Control Ostomy Barrier’ is likely not a ‘control’


In [109]:
# first get only relevant columns from DB
ct_extract = pd.DataFrame(df_dedup[['con_nctid', 'con_downcase_name', 'int_type', 'int_name']])
ct_extract = ct_extract.rename(columns={'con_nctid': 'nctid'})
# get CURIE column for nct_id column (https://bioregistry.io/registry/clinicaltrials)

ct_extract['nctid_curie'] = ct_extract['nctid']
ct_extract['nctid_curie'] = 'clinicaltrials:' + ct_extract['nctid'].astype(str)

In [110]:
ct_extract

,nctid,con_downcase_name,int_type,int_name,nctid_curie
0,NCT03254264,autism spectrum disorder,Behavioral,Early Start Denver Model,clinicaltrials:NCT03254264
1,NCT03254264,autism spectrum disorder,Behavioral,'as-usual' intervention,clinicaltrials:NCT03254264
2,NCT03254329,human milk nutrient reference values,Other,Assessment of human milk nutrient composition,clinicaltrials:NCT03254329
3,NCT03460652,adhd,Drug,KP415 oral capsule,clinicaltrials:NCT03460652
4,NCT03460899,diabetes mellitus with hypoglycemia,Other,Euglycaemic Clamp,clinicaltrials:NCT03460899
...,...,...,...,...,...
972680,NCT03261687,pelvic girdle pain,Other,water based exercise,clinicaltrials:NCT03261687
972681,NCT03261687,pelvic girdle pain,Other,land based exercise,clinicaltrials:NCT03261687
972682,NCT03261622,faecal incontinence,Device,Sacral nerve stimulation at different stimulat...,clinicaltrials:NCT03261622
972683,NCT03261622,fecal incontinence,Device,Sacral nerve stimulation at different stimulat...,clinicaltrials:NCT03261622


In [ ]:
# let's map all the conditions to BTE
# NOTE: we are attempting to match to BTE terms, but many of the conditions terms are not standardized and are misspelled
# First map as many terms as possible to BTE. Whatever is left over, will use other ontologies to create CURIEs


In [ ]:
# recommedation to first format the output as close to sample as possible

In [111]:
final_df = pd.DataFrame(columns=['subject','predicate','object','relation','subject_name','object_name','category'])

In [112]:
# no_col_names_df.columns = col_names_df.columns
# chosen predicate = "related_to", bc "associated_with" implies a statistical relationship 
final_df['subject'] = 'condition:' + ct_extract['con_downcase_name'].astype(str)
final_df['predicate'] = 'biolink:related_to biolink:clinically_demonstrated_to biolink: hypothesized_to'
final_df['object'] = 'intervention:' + ct_extract['int_name'].astype(str)   # this will not all be RxNorm CURIEs since some interventions are not drugs
final_df['relation'] = 'RO:????'
final_df.subject_name = ct_extract.con_downcase_name
final_df.object_name = ct_extract.int_name
final_df.category = 'biolink:Association'
final_df['nctid'] = ct_extract['nctid']
final_df['nctid_curie'] = ct_extract['nctid_curie']


In [113]:
final_df

,subject,predicate,object,relation,subject_name,object_name,category,nctid,nctid_curie
0,condition:autism spectrum disorder,biolink:related_to biolink:clinically_demonstr...,intervention:Early Start Denver Model,RO:????,autism spectrum disorder,Early Start Denver Model,biolink:Association,NCT03254264,clinicaltrials:NCT03254264
1,condition:autism spectrum disorder,biolink:related_to biolink:clinically_demonstr...,intervention:'as-usual' intervention,RO:????,autism spectrum disorder,'as-usual' intervention,biolink:Association,NCT03254264,clinicaltrials:NCT03254264
2,condition:human milk nutrient reference values,biolink:related_to biolink:clinically_demonstr...,intervention:Assessment of human milk nutrient...,RO:????,human milk nutrient reference values,Assessment of human milk nutrient composition,biolink:Association,NCT03254329,clinicaltrials:NCT03254329
3,condition:adhd,biolink:related_to biolink:clinically_demonstr...,intervention:KP415 oral capsule,RO:????,adhd,KP415 oral capsule,biolink:Association,NCT03460652,clinicaltrials:NCT03460652
4,condition:diabetes mellitus with hypoglycemia,biolink:related_to biolink:clinically_demonstr...,intervention:Euglycaemic Clamp,RO:????,diabetes mellitus with hypoglycemia,Euglycaemic Clamp,biolink:Association,NCT03460899,clinicaltrials:NCT03460899
...,...,...,...,...,...,...,...,...,...
972680,condition:pelvic girdle pain,biolink:related_to biolink:clinically_demonstr...,intervention:water based exercise,RO:????,pelvic girdle pain,water based exercise,biolink:Association,NCT03261687,clinicaltrials:NCT03261687
972681,condition:pelvic girdle pain,biolink:related_to biolink:clinically_demonstr...,intervention:land based exercise,RO:????,pelvic girdle pain,land based exercise,biolink:Association,NCT03261687,clinicaltrials:NCT03261687
972682,condition:faecal incontinence,biolink:related_to biolink:clinically_demonstr...,intervention:Sacral nerve stimulation at diffe...,RO:????,faecal incontinence,Sacral nerve stimulation at different stimulat...,biolink:Association,NCT03261622,clinicaltrials:NCT03261622
972683,condition:fecal incontinence,biolink:related_to biolink:clinically_demonstr...,intervention:Sacral nerve stimulation at diffe...,RO:????,fecal incontinence,Sacral nerve stimulation at different stimulat...,biolink:Association,NCT03261622,clinicaltrials:NCT03261622


In [ ]:
final_df.to_csv('ClinTrials_KG_edges.csv', sep ='\t', index=False)


In [ ]:
# interventions will have to be mapped to more than 1 vocabulary unlike conditions, which seems more easily mapped to SNOMED
# first look at the types of interventions (unique on int_type in ct_extract dataframe)

In [114]:
# adding Folder_2/subfolder to the system path
sys.path.insert(0, '/Volumes/TOSHIBA_EXT/ISB/clinical_trials/pymetamap-master')
  
from pymetamap import MetaMap

In [ ]:
# get the directory of the the module
print(os.getcwd())
# Path
path = "/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/bin"  
# Path of Start directory
start = os.getcwd()
print(os.path.relpath("/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/bin", os.getcwd()))

In [115]:
mm = MetaMap.get_instance('/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/bin/metamap18')

In [ ]:
sents = ['breastfeeding adherence', 'this is from a clinical trial in which' ]  # this has to be 1 sentence
concepts,error = mm.extract_concepts(sents)
for concept in concepts:
    print(concept)
    print("\n")

In [ ]:
# check char encoding, is ascii??
    
for ct_string in final_df['subject_name'][:40]:
#     print(type(ct_string))
    print(ct_string.isascii())
    

In [162]:
# MetaMap does not work on non-ascii char, so create dict with what non-ascii char should be replaced with

# diseases = list(set(list(final_df['subject_name'])))
ascii_char_dict = {",": ' ',
                   ';': ' ',
                   '；': ' ',
                   '≥': ' greater than or equal to ',
                   '≤': ' less than or equal to ',
                   '<': ' less than or equal to ',
                   '>': ' less than or equal to ',
                   'ç': 'c',
                   '®': ' ',
                   '©': ' ',
                   'ï': 'i',
                   '̇': '',
                   '"̇': ' ',
#                    '"̇"': ' ',
                   '"': ' ',
                   '\"': ' ',
                   'é': 'e',
                   'è': 'e',
                   'í': 'i',
                   '₁': '1',
                   'ı': '1',
                   '，': ' ',
                   'β': ' beta ',
                   'ß': ' beta ',
                   'ö': 'o',
                   'ō': 'o',
                   '´': "",
                   '²': '2',
                   'γ': ' gamma ',
                   'α': ' alpha ',
                   'ü': 'u',
                   'μg': 'micrograms',
                   'µg': 'micrograms',
                   'µ': 'mu',
                   'μ': 'mu',
                   '•': ' ',
                   '°': ' degrees ',
                   'λ': ' lambda ',
                   'ﬂ': 'fl',
                   'ⅱ': '2',
                   'δ': ' delta ',
#                    'stage i': 'stage 1',
#                    'stage ii': 'stage 2',
#                    'stage iii': 'stage 3',
                   'ⅲ': '3',
                   '（': ' ',
                   '）': ' ',
                   '、': ' ',
                   '¨': ' ',
                   '™': ' ',
                   '：': ' ',
                   'œ': ' ',
                   '／': ' '
                   
                  }

# loop through list of diseases, separate the disease string into char, use dictionary lookup to pick out non-ascii char and replace in string with value from dict    
# diseases = list(filter(None, list(set(list(final_df['subject_name'])))))
diseases = list(filter(None, final_df['subject_name'].unique()))

processed_diseases = [''.join([ascii_char_dict.get(item,item) for item in list(disease)]) for disease in diseases]

# encode any remaining non-ascii characters
diseases_deascii = [disease.encode('ascii', 'ignore').decode('ascii') for disease in processed_diseases]
   
orig_and_process_diseases = pd.DataFrame(list(zip(diseases, diseases_deascii)))
orig_and_process_diseases.columns = ['input_disease', 'processed_disease']
orig_and_process_diseases[:100]


,input_disease,processed_disease
0,autism spectrum disorder,autism spectrum disorder
1,human milk nutrient reference values,human milk nutrient reference values
2,adhd,adhd
3,diabetes mellitus with hypoglycemia,diabetes mellitus with hypoglycemia
4,"diabetes mellitus, type 2",diabetes mellitus type 2
...,...,...
95,tobacco cessation,tobacco cessation
96,tetanus,tetanus
97,osteoarthritis of the knee,osteoarthritis of the knee
98,diabetic nephropathy,diabetic nephropathy


In [117]:
interventions = list(filter(None, final_df['object_name'].unique()))

In [46]:
# for intervention in interventions[:10]:
#     sep_intervention = list(intervention)
#     print(sep_intervention)
    
test_df = final_df[:20]
    
# df.replace(replacements, regex=True, inplace=True)
test_df


    

,subject,predicate,object,relation,subject_name,object_name,category,nctid,nctid_curie
0,condition:autism spectrum disorder,biolink:related_to biolink:clinically_demonstr...,intervention:Early Start Denver Model,RO:????,autism spectrum disorder,Early Start Denver Model,biolink:Association,NCT03254264,clinicaltrials:NCT03254264
1,condition:autism spectrum disorder,biolink:related_to biolink:clinically_demonstr...,intervention:'as-usual' intervention,RO:????,autism spectrum disorder,'as-usual' intervention,biolink:Association,NCT03254264,clinicaltrials:NCT03254264
2,condition:human milk nutrient reference values,biolink:related_to biolink:clinically_demonstr...,intervention:Assessment of human milk nutrient...,RO:????,human milk nutrient reference values,Assessment of human milk nutrient composition,biolink:Association,NCT03254329,clinicaltrials:NCT03254329
3,condition:adhd,biolink:related_to biolink:clinically_demonstr...,intervention:KP415 oral capsule,RO:????,adhd,KP415 oral capsule,biolink:Association,NCT03460652,clinicaltrials:NCT03460652
4,condition:diabetes mellitus with hypoglycemia,biolink:related_to biolink:clinically_demonstr...,intervention:Euglycaemic Clamp,RO:????,diabetes mellitus with hypoglycemia,Euglycaemic Clamp,biolink:Association,NCT03460899,clinicaltrials:NCT03460899
5,condition:diabetes mellitus with hypoglycemia,biolink:related_to biolink:clinically_demonstr...,intervention:Hyperinsulinaemic/Hypoglycaemic C...,RO:????,diabetes mellitus with hypoglycemia,Hyperinsulinaemic/Hypoglycaemic Clamp,biolink:Association,NCT03460899,clinicaltrials:NCT03460899
6,"condition:diabetes mellitus, type 2",biolink:related_to biolink:clinically_demonstr...,intervention:Euglycaemic Clamp,RO:????,"diabetes mellitus, type 2",Euglycaemic Clamp,biolink:Association,NCT03460899,clinicaltrials:NCT03460899
7,"condition:diabetes mellitus, type 2",biolink:related_to biolink:clinically_demonstr...,intervention:Hyperinsulinaemic/Hypoglycaemic C...,RO:????,"diabetes mellitus, type 2",Hyperinsulinaemic/Hypoglycaemic Clamp,biolink:Association,NCT03460899,clinicaltrials:NCT03460899
8,condition:hypoglycemia,biolink:related_to biolink:clinically_demonstr...,intervention:Euglycaemic Clamp,RO:????,hypoglycemia,Euglycaemic Clamp,biolink:Association,NCT03460899,clinicaltrials:NCT03460899
9,condition:hypoglycemia,biolink:related_to biolink:clinically_demonstr...,intervention:Hyperinsulinaemic/Hypoglycaemic C...,RO:????,hypoglycemia,Hyperinsulinaemic/Hypoglycaemic Clamp,biolink:Association,NCT03460899,clinicaltrials:NCT03460899


In [48]:
# interventions = test_df['object_name'].replace(ascii_char_dict, regex=True, inplace=True)
# interventions = test_df.object_name.loc.replace(ascii_char_dict, regex=True, inplace=True)
# df.loc[df.A > 5, 'A'] = 1000
# df.replace({"Courses": dict},inplace=True)
interventions = test_df.replace(ascii_char_dict, inplace=True)


/Users/Kamileh/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [65]:
interventions = list(filter(None, final_df['object_name'].unique()))



In [84]:
import re
l = {'NORTH':'N','SOUTH':'S','EAST':'E','WEST':'W'}
pattern = '|'.join(sorted(re.escape(k) for k in l))
string_test = 'split-virion, non-adjuvanted H1N1 vaccine of 15 μg	split-virion  non-adjuvanted H1N1 vaccine of 15 μg'
re.sub(pattern, lambda m: l.get(m.group(0).upper()), string_test, flags=re.IGNORECASE)
'split-virion, non-adjuvanted H1N1 vaccine of 15 μg	split-virion  non-adjuvanted H1N1 vaccine of 15 μg'

'split-virion, non-adjuvanted H1N1 vaccine of 15 μg\tsplit-virion  non-adjuvanted H1N1 vaccine of 15 μg'

In [87]:
import re
pattern = '|'.join(sorted(re.escape(k) for k in ascii_char_dict))
string_test = 'split-virion, non-adjuvanted H1N1 vaccine of 15 μg'
re.sub(pattern, lambda m: l.get(m.group(0).upper()), string_test, flags=re.IGNORECASE)

'splt-vron non-adjuvanted H1N1 vaccne of 15 g'

In [ ]:
def translate(text, conversion_dict, before=None):
    """
    Translate words from a text using a conversion dictionary

    Arguments:
        text: the text to be translated
        conversion_dict: the conversion dictionary
        before: a function to transform the input
        (by default it will to a lowercase)
    """
    # if empty:
    if not text: return text
    # preliminary transformation:
    before = before or str.lower
    t = before(text)
    for key, value in conversion_dict.items():
        t = t.replace(key, value)
    return t

In [ ]:
translate('hello world', a)

In [89]:
def translate(text, conversion_dict, before=None):
    """
    Translate words from a text using a conversion dictionary

    Arguments:
        text: the text to be translated
        conversion_dict: the conversion dictionary
        before: a function to transform the input
        (by default it will to a lowercase)
    """
    # if empty:
    if not text: return text
    # preliminary transformation:
    before = before or str.lower
    t = before(text)
    for key, value in conversion_dict.items():
        t = t.replace(key, value)
    return t

translate('split-virion, non-adjuvanted H1N1 vaccine of 15 μg', ascii_char_dict)

AttributeError: 'list' object has no attribute 'replace'

In [93]:
test_string = 'split-virion, non-adjuvanted H1N1 vaccine of 15 μg'
for bad_char, replacement in ascii_char_dict.items():
    test_string = test_string.replace(bad_char, replacement)
print(test_string)

split-virion  non-adjuvanted H1N1 vaccine of 15 μg


In [120]:
import re

test_string = 'split-virion, non-adjuvanted H1N1 vaccine of 15 μg'

subs_dict = {"(?<=o)μ":"w", "l(?=a)":"co"}
subs_regex = re.compile("|".join("("+key+")" for key in subs_dict))

group_index = 1
indexed_subs = {}
for target, sub in subs_dict.items():
    indexed_subs[group_index] = sub
    group_index += re.compile(target).groups + 1

test_string = re.sub(subs_regex, lambda match: indexed_subs[match.lastindex], test_string)

In [128]:
final_df.subject_name.to_csv('test_ascii_stripper.csv', sep ='\t', index=False)


In [140]:
mappings = {
    'a':'e',
    'o':'u',
    'i':'eye',
    '!': None
}

tr_table = str.maketrans(ascii_char_dict) 

s = "i absolutely love the american ice-cream!"

print(f"Original string: {s}")

print(f"translation table: {tr_table}")

s1 = s.translate(tr_table) #or str.translate(s, tr_table)

print(f"Translated string: {s1}")

ValueError: string keys in translate table must be of length 1

In [129]:
from __future__ import print_function
import io

with io.open('test_ascii_stripper.csv','r',encoding='utf-8',errors='ignore') as infile, \
     io.open('test_ascii_stripper_out.txt','w',encoding='ascii',errors='ignore') as outfile:
    for line in infile:
        print(*line.split(), file=outfile)

In [ ]:
import re
punct = re.compile("[^\w\s]")
input_file = "test_ascii_stripper.csv"
output_file = "test_ascii_stripper_out.txt"
f_reader = open(input_file)

# writing it to new file
with open(output_file, "w") as f_writer:
    for line in f_reader:
        line = line.strip() #removing whitespace at ends
        line = re.sub(punct, "",line) #removing punctuation
        line += "\n"
        f_writer.write(line)
        
print(f"First 10 lines of original file:")

with open(input_file) as f:
    i = 0
    for line in f:
        print(line,end="")
        i+=1
        if i==10:
            break
            
print(f"\nFirst 10 lines of output file:")

with open(output_file) as f:
    i = 0
    for line in f:
        print(line,end="")
        i+=1
        if i==10:
            break

In [141]:
import re

def custom_make_translation(text, translation):
    regex = re.compile('|'.join(map(re.escape, translation)))
    return regex.sub(lambda match: translation[match.group(0)], text)

translation = {
    "hi":"jk", 
    "as":"bc",
    "c": "i have priority",
    "ca":"sa"
}

text = "hi this is as test ca"
translated = custom_make_translation(text, translation)

In [163]:
import re

def custom_make_translation(text, translation):
    regex = re.compile('|'.join(map(re.escape, translation)))
    return regex.sub(lambda match: translation[match.group(0)], text)

text = "split-virion, non-adjuvanted H1N1 vaccine of 15 μg"
translated = custom_make_translation(text, ascii_char_dict)

In [164]:
translated

'split-virion  non-adjuvanted H1N1 vaccine of 15 micrograms'

In [121]:
test_string

'split-virion, non-adjuvanted H1N1 vaccine of 15 μg'

In [ ]:
# test = final_df['subject_name'].apply(lambda row : custom_make_translation(row['X']))
test = final_df.apply(lambda row: custom_make_translation(row['subject_name'], ascii_char_dict), axis=1)


In [67]:
orig_and_process_interventions = pd.DataFrame(list(zip(interventions, result)))
orig_and_process_interventions.columns = ['input_intervention', 'processed_intervention']

In [70]:
orig_and_process_interventions

,input_intervention,processed_intervention
0,Early Start Denver Model,Early Start Denver Model
1,'as-usual' intervention,'as-usual' intervention
2,Assessment of human milk nutrient composition,Assessment of human milk nutrient composition
3,KP415 oral capsule,KP415 oral capsule
4,Euglycaemic Clamp,Euglycaemic Clamp
...,...,...
350346,medical report consultation,medical report consultation
350347,VAS-FA Turkish version,VAS-FA Turkish version
350348,water based exercise,water based exercise
350349,land based exercise,land based exercise


In [72]:
orig_and_process_interventions.to_csv('processed_interventions_check.csv', sep ='\t', index=False)



In [ ]:
# loop through list of interventions, separate the intervention string into char, use dictionary lookup to pick out non-ascii char and replace in string with value from dict    
processed_interventions = [''.join([ascii_char_dict.get(item,item) for item in list(interventions)]) for intervention in interventions]


In [ ]:
# encode any remaining non-ascii characters
interventions_deascii = [intervention.encode('ascii', 'ignore').decode('ascii') for intervention in processed_interventions]
   
orig_and_process_interventions = pd.DataFrame(list(zip(interventions, interventions_deascii)))
orig_and_process_interventions.columns = ['input_intervention', 'processed_intervention']
orig_and_process_interventions[:100]


In [ ]:
# interventions = final_df['object_name'].unique()
# print(type(interventions))
# len(interventions)


In [ ]:
print(interventions[131405])

In [ ]:
print("how many unique interventions: " + str(len(interventions)))
print("how many unique diseases: " + str(len(diseases)))

In [ ]:
###### COMMENT TO REVERT TO ORIGINAL SIZED DATA SETS, THESE CELLS CREATE SUBSETS OF DATA FOR TESTING PURPOSES

orig_and_process_diseases = orig_and_process_diseases[:200]
orig_and_process_interventions = orig_and_process_interventions[:200]

In [ ]:
with open('processed_diseases_check.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['original_disease','disease_de_ascii'])
    for row in orig_and_process_diseases:
        csv_out.writerow(row)


In [ ]:
# testing metamap on small subsets
diseases_mapped = []

for disease in diseases_deascii[6150:6250]:
    disease = [disease]
    concepts,error = mm.extract_concepts(disease)
#     disease_concepts.append(concepts)
    disease_mapped = [disease, concepts]
    diseases_mapped.append(disease_mapped)

mapped_diseases_df = pd.DataFrame(diseases_mapped)  
mapped_diseases_df

In [ ]:
def run_metamap(orig_and_process_diseases_slice):
    mapped_slice = [] 
    for index, row in orig_and_process_diseases_slice.iterrows():
        disease = [row['processed_disease']]  
        concepts,error = mm.extract_concepts(disease)
        mapped_row = [row['input_disease'], row['processed_disease'], concepts]
        mapped_slice.append(mapped_row)
    return(mapped_slice)
        

mappings = pd.DataFrame(columns=['input_disease', 'processed_disease', 'metamapped_disease'])
for i in range(0, len(orig_and_process_diseases), 100):
# for i in range(0, 200, 10):
    mapped = run_metamap(orig_and_process_diseases[i:i+100])
    mapped = pd.DataFrame(mapped, columns=['input_disease', 'processed_disease', 'metamapped_disease'])
    mappings = mappings.append(mapped)
    print("index at: " + str(i))
  


In [ ]:
mappings.to_pickle('mappings.pkl')


In [ ]:
mappings 


In [ ]:
print(len(mappings))

In [ ]:
# sample_mappings = mappings

In [ ]:
sample_mappings

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
mapped

In [ ]:
# use pooling to multiprocess

import psutil 
print(psutil.cpu_count(logical = False))
print(psutil.cpu_count(logical = True))


In [ ]:
# create batch functions to run

In [ ]:
import multiprocessing

process_list = []
for i in range(2):
    p =  multiprocessing.Process(target=run_metamap)
    p.start()
    process_list.append(p)

for process in process_list:
    process.join()

In [ ]:
# create function to run metamap faster and allow pooling
def run_metamap(orig_and_process_diseases_slice):
    mapped_slice = [] 
    for index, row in orig_and_process_diseases_slice.iterrows():
        disease = [row['processed_disease']]  
        concepts,error = mm.extract_concepts(disease)
        mapped_row = [row['input_disease'], row['processed_disease'], concepts]
        mapped_slice.append(mapped_row)
    return(mapped_slice)
        
def batch_create(orig_and_process_diseases):
    mappings = pd.DataFrame(columns=['input_disease', 'processed_disease', 'metamapped_disease'])
    for i in range(0, len(orig_and_process_diseases), 100):
    # for i in range(0, 200, 10):
        mapped = run_metamap(orig_and_process_diseases[i:i+100])
        mapped = pd.DataFrame(mapped, columns=['input_disease', 'processed_disease', 'metamapped_disease'])
        mappings = mappings.append(mapped)
        print("index at: " + str(i))

In [ ]:
batch_create(orig_and_process_diseases)

In [ ]:

process_list = []
for i in range(2):
    p =  multiprocessing.Process(target=batch_create(orig_and_process_diseases))
    p.start()
    process_list.append(p)

for process in process_list:
    process.join()

In [ ]:
target1 = batch_create(orig_and_process_diseases) 
target2 = batch_create(orig_and_process_interventions) 

In [ ]:
my_list = [i for i in range(10123)]
for i in range(0, len(my_list), 100)
    process_list(my_list[i:i+100])


def process_list(my_list)
    url = 'https://api.example.com'
    data = {'update_list': my_list}
    headers = {'auth': auth}
    r = requests.put(url, data=json.dumps(data), headers=headers)

In [ ]:
# DON'T FORGET TO CHANGE PYMASTER SCRIPT TO OUTPUT SNOMED ONLY TERMS